In [114]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
np.random.seed(42)

In [115]:
df = pd.read_csv('train.csv')
le = preprocessing.LabelEncoder()

In [116]:
df.drop(columns=["PassengerId","Name"],inplace=True,axis=1)

In [117]:
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col].fillna(df[col].mean(),inplace=True)
    if df[col].dtype == object:
        df[col].fillna(df[col].mode()[0],inplace=True)
        df[col] = le.fit_transform(df[col])

In [118]:
y = df["Survived"]
df.drop("Survived",inplace=True,axis=1)
x = df.values

In [119]:
from sklearn.ensemble import RandomForestClassifier
dtc = RandomForestClassifier(n_estimators=200)
dtc.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [120]:
#Training Accuracy
y_pred = dtc.predict(x)
y_pred = np.array(y_pred)
y_train = np.array(y)
print(metrics.classification_report(y_train, y_pred))
print(metrics.confusion_matrix(y_train, y_pred))
print(metrics.accuracy_score(y_train,y_pred)*100)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       1.00      1.00      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891

[[548   1]
 [  0 342]]
99.88776655443323


# Testing Phase

In [121]:
df = pd.read_csv('test.csv')
passenger_id = np.array(df.PassengerId)

In [122]:
df.drop(columns=["PassengerId","Name"],inplace=True,axis=1)

In [123]:
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col].fillna(df[col].mean(),inplace=True)
    if df[col].dtype == object:
        df[col].fillna(df[col].mode()[0],inplace=True)
        df[col] = le.fit_transform(df[col])

In [124]:
x_test = df.values

In [125]:
#Testing Accuracy
y_pred = dtc.predict(x_test)
y_pred = np.array(y_pred)
print(np.unique(y_pred,return_counts=True))
values = np.vstack((passenger_id, y_pred)).transpose()

(array([0, 1]), array([255, 163]))


In [126]:
prediction_df = pd.DataFrame(values,columns=["PassengerId","Survived"])

In [127]:
prediction_df.to_csv('out.csv',index=False)